# Расчёт SEM, MDE и мощности

В ноутбуке мы разберём:

- **Стандартная ошибка среднего (SEM)** — как оценить точность измерения среднего, и чем она отличается для **бинарных** и **непрерывных** метрик.

-  **Minimum Detectable Effect (MDE)**  — минимальный эффект, который можно обнаружить, как он **рассчитывается вручную** и **через функции**.

- **Распределения и мощность теста**  — визуализируем, как работают **нулевая и альтернативная гипотезы**, где проходит **критическая граница** и что значит **80% мощности**.

- **Зависимость MDE от объёма выборки** —  **сколько трафика нужно**, чтобы обнаружить эффект в 2%, и как MDE уменьшается с ростом `n`

## Стандартная ошибка среднего (SEM) для разных типов данных

В A/B-тестировании важно понимать, как оценивать **точность среднего значения**, измеренного по выборке. Для этого используется **стандартная ошибка среднего (Standard Error of the Mean, SEM)**.

Однако формула для SEM **зависит от типа метрики**:
- **Бинарные данные** (например, конверсия)
- **Непрерывные данные** (например, средний чек, время на сайте)

Рассмотрим оба случая.

---

## SEM для бинарных данных (доля успехов)

Когда метрика — **доля положительных исходов** (например, конверсия), каждый пользователь даёт результат 0 (не конвертировался) или 1 (конвертировался).

Такая величина следует **распределению Бернулли**, а сумма успехов — **биномиальному распределению**.

###  Формула:
$$
\text{SEM} = \sqrt{\frac{p(1 - p)}{n}}
$$

Где:
- $ p $ — доля успехов (например, конверсия)
- $ n $ — размер выборки


- Дисперсия одного наблюдения (Бернулли): $ \sigma^2 = p(1 - p) $
- Дисперсия среднего: $ \frac{\sigma^2}{n} = \frac{p(1 - p)}{n} $
- Стандартная ошибка: корень из дисперсии среднего

 Это **точная формула**, а не приближение.

---

## SEM для непрерывных данных

Когда метрика — **непрерывная величина** (например, средний чек, время на сайте, CPM), мы не знаем истинного стандартного отклонения и оцениваем его по выборке.

### Формула:
$$
\text{SEM} = \frac{s}{\sqrt{n}}
$$

Где:
- $ s $ — выборочное стандартное отклонение
- $ n $ — размер выборки

### Как рассчитывается $ s $?
$$
s = \sqrt{\frac{1}{n - 1} \sum_{i=1}^{n} (x_i - \bar{x})^2}
$$

Здесь используется **исправленное стандартное отклонение** (с $ n-1 $ в знаменателе), чтобы уменьшить смещение оценки.

---

## Сравнение двух случаев

| Характеристика | Бинарные данные (конверсия) | Непрерывные данные (средний чек) |
|----------------|-----------------------------|----------------------------------|
| Пример | Кликал / не кликал | Сумма покупки (в рублях) |
| Распределение | Бернулли / Биномиальное | Нормальное, логнормальное и др. |
| Среднее | $ \hat{p} = \frac{\text{число успехов}}{n} $ | $ \bar{x} = \frac{1}{n}\sum x_i $ |
| Дисперсия | $ p(1 - p) $ | Оценивается по данным: $ s^2 $ |
| SEM | $ \sqrt{\frac{p(1 - p)}{n}} $ | $ \frac{s}{\sqrt{n}} $ |
| Масштаб | Безразмерный (0–1) | Имеет единицы измерения (например, рубли) |


### Вывод

- Для **бинарных метрик** (конверсия, CTR) используем:  
  $$
  \text{SEM} = \sqrt{\frac{p(1 - p)}{n}}
  $$

- Для **непрерывных метрик** (средний чек, время на сайте) используем:  
  $$
  \text{SEM} = \frac{s}{\sqrt{n}}
  $$

Создадим универсальную функцию для расчёта SEM, которая будет работать как для бинарных данных (конверсия), так и для непрерывных (например, средний чек, время на сайте и т.д.).

In [4]:
import numpy as np
from typing import Union, Optional

def calculate_sem(
    data: Optional[Union[list, np.ndarray]] = None,
    p: Optional[float] = None,
    n: Optional[int] = None,
    data_type: str = 'auto'
) -> float:
    """
    Универсальный расчёт Standard Error of the Mean (SEM).
    
    Поддерживает:
    - Бинарные данные (0/1): SEM = sqrt(p*(1-p)/n)
    - Непрерывные данные: SEM = std / sqrt(n)
    
    Параметры:
    ----------
    data : array-like, optional
        Массив данных (0/1 или непрерывные). Если передан — используется для расчёта.
    p : float, optional
        Оценка доли (для бинарных данных), если данные не переданы. Должно быть в [0, 1].
    n : int, optional
        Размер выборки, если данные не переданы.
    data_type : {'auto', 'binary', 'continuous'}
        Тип данных. При 'auto' определяется автоматически.
    
    Возвращает:
    -----------
    float : Standard Error of the Mean (SEM)
    
    Примеры:
    --------
    >>> calculate_sem(data=[0,1,1,0,1])
    0.2236
    
    >>> calculate_sem(p=0.5, n=100)
    0.05
    
    >>> calculate_sem(data=[1.2, 2.3, 1.8, 3.1], data_type='continuous')
    0.3715
    
    Raises:
    -------
    ValueError : если параметры некорректны
    TypeError : если типы не соответствуют ожиданиям
    """
    
    # --- Валидация и приведение типов ---
    if data is not None:
        try:
            data = np.asarray(data, dtype=float)
        except (ValueError, TypeError):
            raise TypeError("Данные должны быть числовыми (0, 1 или вещественные).")

        if data.size == 0:
            raise ValueError("Массив данных пуст.")

        n_data = data.size

        # Автоопределение типа
        if data_type == 'auto':
            unique_vals = np.unique(data)
            if np.all(np.isin(unique_vals, [0.0, 1.0])):
                data_type = 'binary'
            else:
                data_type = 'continuous'

        # Бинарные данные
        if data_type == 'binary':
            p_hat = np.mean(data)
            if p_hat == 0 or p_hat == 1:
                # При p=0 или p=1 SEM=0, но может быть нестабильно
                return 0.0
            sem = np.sqrt(p_hat * (1 - p_hat) / n_data)
            return float(sem)

        # Непрерывные данные
        elif data_type == 'continuous':
            if n_data == 1:
                return 0.0  # SEM = 0 при одном наблюдении
            std = np.std(data, ddof=1)  # исправленное стандартное отклонение
            sem = std / np.sqrt(n_data)
            return float(sem)

        else:
            raise ValueError("data_type должен быть 'binary', 'continuous' или 'auto'.")

    # --- Расчёт по p и n ---
    elif p is not None and n is not None:
        if not isinstance(n, (int, np.integer)) or n <= 0:
            raise ValueError("n должно быть положительным целым числом.")
        if not isinstance(p, (float, int)) or not (0 <= p <= 1):
            raise ValueError("p должно быть числом в диапазоне [0, 1].")

        if p == 0 or p == 1:
            return 0.0  # крайние случаи

        sem = np.sqrt(p * (1 - p) / n)
        return float(sem)

    else:
        raise ValueError(
            "Нужно передать либо `data`, либо `p` и `n` одновременно."
        )

### Пример 1: Бинарные данные (конверсия)

In [5]:
# Пример: 100 конверсий из 1000 пользователей
np.random.seed(42)
data_binary = np.random.binomial(1, 0.10, 1000)  # 1000 пользователей, p=0.1

sem_binary = calculate_sem(data=data_binary)
print(f"SEM (бинарные данные) = {sem_binary:.6f}")

SEM (бинарные данные) = 0.009487


### Пример 2: Непрерывные данные (например, средний чек)

In [6]:
# Пример: средний чек в магазине (в рублях)
np.random.seed(42)
data_continuous = np.random.lognormal(mean=4, sigma=1.0, size=1000)  # имитация чеков

sem_continuous = calculate_sem(data=data_continuous)
print(f"Средний чек = {np.mean(data_continuous):.2f} руб.")
print(f"SEM (непрерывные данные) = {sem_continuous:.6f}")

Средний чек = 91.85 руб.
SEM (непрерывные данные) = 4.223391


### Пример 3: Только p и n (без данных)

In [7]:
sem_from_p_n = calculate_sem(p=0.10, n=1000)
print(f"SEM (по p и n) = {sem_from_p_n:.6f}")

SEM (по p и n) = 0.009487



## Minimum Detectable Effect — минимальный обнаруживаемый эффект


**MDE (Minimum Detectable Effect)** — это **наименьшее изменение в метрике**, которое можно **статистически значимо обнаружить** при заданных условиях:
- размер выборки (`n`)
- уровень значимости (`α`)
- мощность теста (`1−β`)
- базовое значение метрики (`p₀`)

> *«Если эффект меньше MDE — мы, скорее всего, его не заметим, даже если он реальный».*


###  Формула MDE (для пропорций)

Для A/B-тестов с бинарными метриками (например, конверсия):

$$
\text{MDE} = \text{SEM} \times (Z_{1-\alpha/2} + Z_{1-\beta})
$$

Где:
- $\text{SEM} = \sqrt{\frac{p_0 (1 - p_0)}{n}}$ — стандартная ошибка среднего
- $Z_{1-\alpha/2}$ — критическое значение нормального распределения (при $\alpha=0.05$  $1.96$)
- $Z_{1-\beta}$ — критическое значение для мощности (при $\text{power}=0.8$  $0.84$)

---

### Откуда берутся значения $ Z_{1-\alpha/2} = 1.96 $ и $ Z_{1-\beta} = 0.84 $

Эти числа — **квантили стандартного нормального распределения**.  
Они определяются на основе **уровня значимости** и **мощности теста**, которые мы задаём перед A/B-тестом.

---

###  $ Z_{1-\alpha/2} = 1.96 $

### Что означает:
- $ \alpha = 0.05 $ — вероятность **ошибки I рода** (ложноположительный результат)
- Мы используем **двухсторонний тест**, поэтому делим $ \alpha $ на два хвоста: $ \alpha/2 = 0.025 $
- Нам нужно найти $ Z $ (из таблицы стандартного нормального распределения или stats.norm.ppf из python), при котором площадь слева = $ 1 - 0.025 = 0.975 $

$$
Z_{1 - \alpha/2} = Z_{0.975} \approx 1.96 
$$  

Это означает:  
> 95% значений стандартного нормального распределения лежат в интервале $[-1.96, +1.96]$.

---

###  $ Z_{1-\beta} = 0.84 $

### Что означает:
- $ \beta = 0.2 $ — вероятность **ошибки II рода** (пропустить реальный эффект)
- $ 1 - \beta = 0.8 $ — **мощность теста** (вероятность обнаружить эффект, если он есть)
- Нам нужно найти $ Z $, при котором площадь слева = $ 0.8 $

$$
Z_{1 - \beta} = Z_{0.8} \approx 0.84
$$

> Чтобы обнаружить эффект с вероятностью 80%, критическая граница должна быть на $ 0.84\sigma $ правее среднего при альтернативной гипотезе.

---

###  Таблица: часто используемые Z-значения

| Параметр | Уровень | $ Z $-значение |
|--------|--------|----------------|
| $ \alpha = 0.05 $ (двухсторонний) | $ 1 - \alpha/2 = 0.975 $ | $ 1.96 $ |
| $ \alpha = 0.01 $ | $ 0.995 $ | $ 2.58 $ |
| $ \text{power} = 0.8 $ | $ 0.80 $ | $ 0.84 $ |
| $ \text{power} = 0.9 $ | $ 0.90 $ | $ 1.28 $ |
| $ \text{power} = 0.95 $ | $ 0.95 $ | $ 1.645 $ |

---

### Почему именно сумма $ Z_{1-\alpha/2} + Z_{1-\beta} $?

Потому что:

- Расстояние между центром **нулевой гипотезы** (H₀) и **критической границей** = $ Z_{1-\alpha/2} \cdot \text{SEM} $
- Расстояние между центром **альтернативной гипотезы** (H₁) и той же границей = $ Z_{1-\beta} \cdot \text{SEM} $
- Общее расстояние между H₀ и H₁ — это и есть **MDE**

$$
\text{MDE} = \text{SEM} \times (Z_{1-\alpha/2} + Z_{1-\beta})
$$

---


| Обозначение | Значение | Откуда берётся |
|------------|--------|----------------|
| $ Z_{1-\alpha/2} $ | $ 1.96 $ | Квантиль $ N(0,1) $ на уровне $ 0.975 $ |
| $ Z_{1-\beta} $ | $ 0.84 $ | Квантиль $ N(0,1) $ на уровне $ 0.80 $ |

Эти значения **универсальны** для A/B-тестов с:
- уровнем значимости $ \alpha = 0.05 $
- мощностью $ 80\% $

Их не нужно запоминать — достаточно знать, **как они рассчитываются**.

### Пример

Пусть:
- Базовая конверсия: $p_0 = 0.10$ (10%)
- Размер выборки: $n = 1000$
- $\alpha = 0.05$, $\text{power} = 0.8$

Тогда:
- $\text{SEM} = \sqrt{\frac{0.10 \times 0.90}{1000}} = 0.009487$
- $Z_{1-\alpha/2} = 1.96$, $Z_{1-\beta} = 0.84$
- $\text{MDE} = 0.009487 \times (1.96 + 0.84) = 0.009487 \times 2.80 = 0.0266$

**MDE = 2.66%**

---

### Интерпретация

При выборке в **1000 пользователей** мы сможем обнаружить **только эффекты не менее +2.66%** к конверсии.

- Если тест даст рост до **12%** → эффект **не будет статистически значим**
- Если тест даст рост до **13%** → эффект, скорее всего, **будет обнаружен**

> ❗ MDE — это **не ожидаемый эффект**, а **порог чувствительности теста**.

---

### Как уменьшить MDE?

Чтобы обнаруживать **меньшие эффекты**, нужно:
- **Увеличить размер выборки** → снижает SEM
- **Повысить базовую конверсию** (если возможно)
- Снизить мощность или ослабить α — **не рекомендуется**

---

### Распространённые заблуждения

| Заблуждение | Правда |
|------------|-------|
| «MDE — это ожидаемый эффект» | Нет, это **минимальный обнаружимый**, а не ожидаемый |
| «Если эффект < MDE — его нет» | Нет, он может быть — но мы **не сможем его подтвердить** |
| «Чем меньше MDE — тем лучше» | Да, но: **меньший MDE = больше трафика и времени** |

---

- MDE показывает, **насколько чувствителен ваш A/B-тест**
- Перед запуском теста **всегда проверяйте MDE** — иначе можно потратить время зря
- Если **целевой эффект меньше MDE** — увеличьте выборку или пересмотрите гипотезу

In [8]:
# Теперь используем SEM для расчета MDE
from scipy import stats

p0 = 0.10
n = 1000
alpha = 0.05
power = 0.8

# Рассчитаем SEM
SEM = calculate_sem(p=p0, n=n)

# Z-значения
z_alpha = stats.norm.ppf(1 - alpha/2)  # 1.96
z_beta = stats.norm.ppf(power)         # 0.84

# MDE
MDE = SEM * (z_alpha + z_beta)
print(f"MDE = {SEM:.6f} × ({z_alpha:.4f} + {z_beta:.4f}) = {MDE:.6f}")

MDE = 0.009487 × (1.9600 + 0.8416) = 0.026578


In [9]:
# Для бинарных данных
p_hat = np.mean(data_binary)
sem_formula = np.sqrt(p_hat * (1 - p_hat) / len(data_binary))
print(f"SEM (данные) = {sem_binary:.6f}")
print(f"SEM (формула) = {sem_formula:.6f}")

SEM (данные) = 0.009487
SEM (формула) = 0.009487


In [10]:
# Для непрерывных
std_data = np.std(data_continuous, ddof=1)
sem_manual = std_data / np.sqrt(len(data_continuous))
print(f"SEM (данные) = {sem_continuous:.6f}")
print(f"SEM (ручной) = {sem_manual:.6f}")

SEM (данные) = 4.223391
SEM (ручной) = 4.223391


---

В предыдущих ячейках:
- В **бинарных данных** SEM ≈ `0.0095` - это **ошибка оценки доли** (например, конверсии 10%)
- В **непрерывных данных** SEM ≈ `4.22` - это **ошибка оценки среднего чека** (в рублях)

Они **не должны совпадать** - это **разные метрики с разной природой и масштабом**.

---

In [11]:
!pip install plotly numpy pandas scipy

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install --upgrade plotly

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats

In [26]:
p0 = 0.10        # базовая конверсия (10%)
n = 1000         # размер выборки в одной группе
alpha = 0.05     # уровень значимости
beta = 0.2       # ошибка II рода
power = 1 - beta # мощность = 80%
mde_target = 0.02

In [15]:
## РУЧНОЙ РАСЧЁТ ПО ФОРМУЛЕ

# Шаг 1: SEM (Standard Error of the Mean)
SEM = np.sqrt(p0 * (1 - p0) / n)

# Шаг 2: Z-значения
Z_alpha = stats.norm.ppf(1 - alpha/2)  # двухсторонний: ≈ 1.96
Z_beta  = stats.norm.ppf(1 - beta)     # для мощности: ≈ 0.84

# Шаг 3: MDE по формуле
MDE_manual = SEM * (Z_alpha + Z_beta)

# Шаг 4: m и m₀
m0 = p0
m = m0 + MDE_manual

# Вывод
print(" РУЧНОЙ РАСЧЁТ ПО ФОРМУЛЕ:")
print(f"SEM = sqrt({p0} × {1-p0} / {n}) = {SEM:.6f}")
print(f"Z_{{1-α/2}} = Z_{{0.975}} = {Z_alpha:.4f}")
print(f"Z_{{1-β}} = Z_{{0.8}} = {Z_beta:.4f}")
print(f"MDE = {SEM:.6f} × ({Z_alpha:.4f} + {Z_beta:.4f}) = {MDE_manual:.6f}")
print(f"m₀ = {m0:.6f}")
print(f"m  = {m:.6f}")
print(f"→ m - m₀ = {m - m0:.6f}")
print(f" Все значения совпадают: MDE = m - m₀ = {MDE_manual:.6f}")

 РУЧНОЙ РАСЧЁТ ПО ФОРМУЛЕ:
SEM = sqrt(0.1 × 0.9 / 1000) = 0.009487
Z_{1-α/2} = Z_{0.975} = 1.9600
Z_{1-β} = Z_{0.8} = 0.8416
MDE = 0.009487 × (1.9600 + 0.8416) = 0.026578
m₀ = 0.100000
m  = 0.126578
→ m - m₀ = 0.026578
 Все значения совпадают: MDE = m - m₀ = 0.026578


In [16]:
## РАСЧЁТ ЧЕРЕЗ ФУНКЦИЮ

def calculate_mde(
    p0: float,
    n: int,
    alpha: float = 0.05,
    power: float = 0.8
) -> float:
    """
    Рассчитывает Minimum Detectable Effect (MDE) для A/B-теста с бинарной метрикой.
    
    Использует универсальную функцию calculate_sem для корректного расчёта SEM.
    
    Параметры:
    ----------
    p0 : float
        Базовая конверсия (доля в контрольной группе), от 0 до 1.
    n : int
        Размер выборки в одной группе.
    alpha : float, default=0.05
        Уровень значимости (обычно 0.05).
    power : float, default=0.8
        Мощность теста (обычно 0.8).
    
    Возвращает:
    -----------
    float : MDE (в абсолютных единицах)
    """
    # Используем универсальную функцию для расчёта SEM
    SEM = calculate_sem(p=p0, n=n)
    
    # Критические значения нормального распределения
    z_alpha = stats.norm.ppf(1 - alpha / 2)  # двухсторонний тест
    z_beta = stats.norm.ppf(power)           # для мощности (1 - beta)
    
    # MDE = SEM × (Z_α + Z_β)
    MDE = SEM * (z_alpha + z_beta)
    
    return MDE

MDE_function = calculate_mde(p0, n, alpha, power)

print(" РАСЧЁТ ЧЕРЕЗ ФУНКЦИЮ (с calculate_sem):")
print(f"MDE (функция) = {MDE_function:.6f}")

 РАСЧЁТ ЧЕРЕЗ ФУНКЦИЮ (с calculate_sem):
MDE (функция) = 0.026578


In [17]:
print(" СРАВНЕНИЕ РЕЗУЛЬТАТОВ:")
print(f"Ручной расчёт: {MDE_manual:.6f}")
print(f"Через функцию: {MDE_function:.6f}")
print(f"Совпадают: {np.isclose(MDE_manual, MDE_function)}")

 СРАВНЕНИЕ РЕЗУЛЬТАТОВ:
Ручной расчёт: 0.026578
Через функцию: 0.026578
Совпадают: True


In [28]:
## РАСЧЕТ НЕОБХОДИМОГО РАЗМЕРА ВЫБОРКИ

def calculate_sample_size(p0, mde, alpha=0.05, power=0.8):
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    numerator = (z_alpha + z_beta)**2 * p0 * (1 - p0)
    denominator = mde**2
    return int(np.ceil(numerator / denominator))


n_required = calculate_sample_size(p0, mde_target, alpha, power)

print(f" Чтобы обнаружить MDE = {mde_target:.4f} ({mde_target*100:.1f}%), нужно n = {n_required} в каждой группе")

 Чтобы обнаружить MDE = 0.0200 (2.0%), нужно n = 1766 в каждой группе


In [30]:
# Пример: хотим обнаружить эффект 2,65% ( 0.026578)
n_required = calculate_sample_size(p0, mde_target, alpha, power)
mde_target =  0.026578
print(f" Чтобы обнаружить MDE = {mde_target:.4f} ({mde_target*100:.1f}%), нужно n = {n_required} в каждой группе")

 Чтобы обнаружить MDE = 0.0266 (2.7%), нужно n = 1001 в каждой группе


### Почему небольшое уменьшение MDE требует почти в 2 раза больше данных?

- Чтобы обнаружить **MDE = 2.7%** → нужно **1001 пользователь** в группе  
- Чтобы обнаружить **MDE = 2.0%** → нужно **1766 пользователей** в группе

Уменьшение MDE всего на **0.7 п.п.** увеличивает объём выборки на **76%** — почти в 2 раза!

Потому что **размер выборки обратно пропорционален квадрату MDE**:

$$
n \propto \frac{1}{\text{MDE}^2}
$$
Это **не линейная**, а **квадратичная зависимость**.

####  Посчитаем:
$$
\left( \frac{0.0266}{0.0200} \right)^2 = (1.33)^2 = 1.77
$$

Значит, объём выборки должен вырасти в **1.77 раза**  
$ 1001 \times 1.77 \approx 1772 $ → очень близко к **1766**

Чем **меньше эффект**, тем **труднее его обнаружить**, и тем **быстрее растёт нужный объём данных**.

> Представь, что нужно найти иголку в стоге сена:
> - Если иголка **толстая** (MDE = 2.7%) — её легко заметить, достаточно маленького стога
> - Если иголка **тонкая** (MDE = 2.0%) — нужно перебрать **гораздо больше сена**, чтобы быть уверенным


## Почему это важно?

### 1. **Интуиция обманывает**
- Кажется: "2.7% и 2.0% — почти одинаково"
- На самом деле: разница в **чувствительности теста** огромна
- Переход с 2.7% на 2.0% — это переход от "обнаруживаю только крупные эффекты" к "могу видеть мелкие"

### 2. **Стоимость теста растёт нелинейно**
- При $ n = 1001 $: тест можно завершить за **1–2 недели**
- При $ n = 1766 $: нужно **на 76% больше трафика** → может потребоваться **в 2 раза больше времени**
- На низкотрафиковых сайтах — тест может стать **невозможным**

### 3. **Риск "теста впустую"**
Если не рассчитать MDE заранее:
- Ты думаешь: "Хочу увидеть +2%"
- А при текущем трафике MDE = 2.7%
- Даже если эффект реальный (например, +2.5%) — тест **не покажет значимости**
- Вывод: "нет эффекта", хотя он был

> ❗ Это одна из **самых частых ошибок** в A/B-тестировании.

### 4. **MDE — это компромисс**
- Хочешь **высокую чувствительность**? → жертвуй временем
- Хочешь **быстро получить результат**? → соглашайся на **меньшую точность**
- Нужно всегда **балансировать**


## Вывод

> ❗ **Даже небольшое уменьшение MDE (например, с 2.7% до 2.0%) требует почти в 2 раза больше данных**, потому что:
> - MDE стоит в **знаменателе в квадрате**
> - Зависимость — **нелинейная**
>
> Это **не ошибка, а закон статистики**
>
> И это **критически важно**, потому что:
> - без учёта MDE можно **потратить месяц на бесполезный тест**
> - правильно спланированный тест **даёт уверенность в результатах**


Перед запуском A/B-теста **всегда** важен вопрос:

> *"Какой минимальный эффект мне важен?"*  
> Это и будет **целевой MDE**  
> По нему рассчитывается **нужный объём выборки**

И только потом принимается решение **стоит ли запускать этот тест?**

In [23]:
## РАСПРЕДЕЛЕНИЕ КОНТРОЛЬНОЙ И ТЕСТОВОЙ ГРУПП

x = np.linspace(0.05, 0.15, 200)
y_control = stats.norm.pdf(x, p0, SEM)
y_test = stats.norm.pdf(x, p0 + MDE_manual, SEM)

critical_upper = p0 + Z_alpha * SEM

fig1 = go.Figure()

# Контрольная группа (A)
fig1.add_trace(go.Scatter(
    x=x, y=y_control,
    mode='lines',
    name='Контроль (A)',
    line=dict(color='blue')
))

# Тестовая группа при MDE
fig1.add_trace(go.Scatter(
    x=x, y=y_test,
    mode='lines',
    name='Тест (B) при MDE',
    line=dict(color='orange')
))

# Вертикальная линия — критическая граница
fig1.add_shape(
    type='line',
    x0=critical_upper, x1=critical_upper,
    y0=0, y1=1,
    xref='x',
    yref='paper',  # Правильно: 'paper', а не 'y domain'
    line=dict(color='red', dash='dash', width=2)
)

# Подпись к линии
fig1.add_annotation(
    x=critical_upper,
    y=0.95,
    xref='x',
    yref='paper',
    text="Критическая граница",
    showarrow=False,
    font=dict(size=12, color="black"),
    bgcolor="white",
    opacity=0.8
)

# Заштрихованная область — без видимой линии
x_fill = x[x >= critical_upper]
y_fill = stats.norm.pdf(x_fill, p0 + MDE_manual, SEM)
fig1.add_trace(go.Scatter(
    x=x_fill,
    y=y_fill,
    fill='tozeroy',
    fillcolor='rgba(255, 165, 0, 0.3)',
    line=dict(width=0), 
    name='Область обнаружения'
))

fig1.update_layout(
    title="Распределения контрольной и тестовой групп",
    xaxis_title="Конверсия",
    yaxis_title="Плотность вероятности",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(x=0.7, y=0.9)
)

fig1.show()

### Интерпретация графика

Этот график 
- Показывает, **почему нужен MDE**
- Как работает **уровень значимости**
- Что такое **мощность теста**
- Почему **не всегда можно обнаружить маленький эффект**

Визуализирует **статистическую мощность A/B-теста** при заданных параметрах:
- Базовая конверсия: $ p_0 = 0.10 $
- Размер выборки: $ n = 1000 $
- Уровень значимости: $ \alpha = 0.05 $
- Мощность теста: $ 80\% $


| Кривая | Что означает |
|--------|-------------|
| 🔵 Синяя кривая (Контроль) | Распределение среднего значения в **контрольной группе** при нулевой гипотезе ($ H_0 $): конверсия = 0.10 |
| 🟠 Оранжевая кривая (Тест при MDE) | Распределение среднего значения в **тестовой группе** при альтернативной гипотезе ($ H_1 $): конверсия = $ p_0 + \text{MDE} = 0.1266 $ |

> Эти кривые — это **нормальные распределения** с одинаковой дисперсией (SEM), но разными средними.

---

### Критическая граница (красная пунктирная линия)

- Позиция: $ x = 0.1183417 $
- Формула: $ \text{critical\_upper} = p_0 + Z_{1-\alpha/2} \times \text{SEM} $
- $ Z_{1-\alpha/2} = 1.96 $, $ \text{SEM} = 0.009487 $
- $ 0.10 + 1.96 \times 0.009487 = 0.1186 $ → округлено до **0.1183**

Это **граница решений**: если среднее в тестовой группе > 0.1183 — эффект считается **статистически значимым**.

---

### Область обнаружения (закрашенная оранжевым)

- Это **площадь под оранжевой кривой справа от критической границы**
- Она равна **мощности теста** — ~80%
- Значит: если реальный эффект = MDE, то с вероятностью 80% мы его **обнаружим**

> Если эффект меньше MDE — эта площадь уменьшается, и риск пропустить эффект растёт.

---

### Числа в подписи (hover)

| Значение | Что означает |
|--------|-------------|
| `Контроль (A): 6.487787` | Плотность вероятности в точке $ x = 0.1183 $ для контрольной группы |
| `Тест (B): 28.8476` | Плотность вероятности в той же точке для тестовой группы |
| `0.1183417` | Координата критической границы |

> Плотность не является вероятностью, но показывает, насколько "плотно" распределены значения около этой точки.

---

### Почему MDE = 0.0266?

- $ \text{MDE} = \text{SEM} \times (Z_{1-\alpha/2} + Z_{1-\beta}) $
- $ \text{SEM} = 0.009487 $
- $ Z_{1-\alpha/2} = 1.96 $, $ Z_{1-\beta} = 0.84 $
- $ \text{MDE} = 0.009487 \times (1.96 + 0.84) = 0.0266 $

На графике: расстояние между пиком синей кривой (0.10) и пиком оранжевой (0.1266) = **0.0266**

In [20]:
# MDE И ОБЪЕМ ВЫБОРКИ

n_values = np.arange(200, 5000, 100)
mde_values = [calculate_mde(p0, n, alpha, power) for n in n_values]

fig2 = go.Figure()

# Линия MDE
fig2.add_trace(go.Scatter(
    x=n_values,
    y=mde_values,
    mode='lines+markers',
    name='MDE',
    line=dict(color='green')
))

# Горизонтальная линия - целевой MDE
fig2.add_shape(
    type='line',
    x0=n_values[0], x1=n_values[-1],
    y0=mde_target, y1=mde_target,
    xref='x', yref='y',
    line=dict(color='purple', dash='dot', width=2)
)

# Подпись к горизонтальной линии
fig2.add_annotation(
    x=n_values[-1] - 200,
    y=mde_target,
    xref='x', yref='y',
    text=f"Целевой MDE: {mde_target*100:.1f}%",
    showarrow=False,
    font=dict(size=12, color="purple"),
    bgcolor="white",
    opacity=0.8
)

# 🔵 Найдём ТОЧНОЕ n, при котором MDE = mde_target
z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta = stats.norm.ppf(power)
numerator = (z_alpha + z_beta)**2 * p0 * (1 - p0)
denominator = mde_target**2
n_exact = numerator / denominator

# Округлим до целого
n_exact_rounded = int(np.ceil(n_exact))

# Добавим точку пересечения
fig2.add_trace(go.Scatter(
    x=[n_exact_rounded],
    y=[mde_target],
    mode='markers',
    marker=dict(
        color='black',
        size=10,
        symbol='circle',
        line=dict(width=2, color='white')
    ),
    name='Точка пересечения'
))

# Подпись сверху
fig2.add_annotation(
    x=n_exact_rounded,
    y=mde_target + 0.0001,
    xref='x', yref='y',
    text=f'n = {n_exact_rounded}',
    showarrow=True,
    arrowhead=4,
    ax=0, ay=-20,
    font=dict(size=12, color="black"),
    bgcolor="white",
    opacity=0.9
)

# Вертикальная пунктирная линия от точки до оси X
fig2.add_shape(
    type='line',
    x0=n_exact_rounded, x1=n_exact_rounded,
    y0=0, y1=1,
    xref='x', yref='paper',
    line=dict(color='black', dash='dash', width=1)
)

fig2.update_layout(
    title="Как MDE уменьшается с ростом объёма выборки",
    xaxis_title="Размер выборки (n)",
    yaxis_title="MDE (абсолютное значение)",
    hovermode="x unified",
    template="plotly_white"
)

fig2.show()

### Интерпретация графика

Этот график показывает, **как минимальный обнаруживаемый эффект (MDE)** зависит от **размера выборки (n)** при фиксированных:
- базовой конверсии $ p_0 = 0.10 $
- уровне значимости $ \alpha = 0.05 $
- мощности теста $ \text{power} = 0.8 $
- **сколько трафика нужно** для достижения целевого MDE
- Помогает **избежать ошибок**:
  - слишком малой выборки (нельзя обнаружить эффект)
  - слишком большой выборки 


| Элемент | Что означает |
|--------|-------------|
| 🟢 Зелёная кривая | MDE как функция от $ n $. Чем больше $ n $, тем меньше MDE |
| 🟣 Пунктирная линия | Целевой MDE = 2.0% (абсолютное значение) |
| ⚫ Точка пересечения | При $ n = 1766 $, MDE ≈ 2.0% → **достаточный объём выборки** |

> Если $ n < 1766 $, MDE > 2.0% → тест не чувствителен к малым эффектам  
> Если $ n > 1766 $, MDE < 2.0% → тест может обнаружить даже небольшие изменения


- Чтобы **обнаружить эффект 2%**, нужно **не менее 1766 пользователей в каждой группе**
- При $ n = 500 $: MDE ≈ 5% → тест не подходит для малых эффектов
- При $ n = 5000 $: MDE ≈ 1.1% → тест очень чувствителен

> Но: **больше данных = больше времени и затрат**  
> Поэтому всегда выбирай **оптимальный размер выборки**: достаточно, чтобы обнаружить нужный эффект, но не больше.


Кривая имеет форму гиперболы потому что:
- $ \text{SEM} \propto \frac{1}{\sqrt{n}} $
- А MDE пропорционален SEM
- Значит: $ \text{MDE} \propto \frac{1}{\sqrt{n}} $
Это **обратно пропорциональная зависимость**, поэтому кривая плавная и не достигает нуля.

In [21]:
sample_sizes = np.arange(2000, 5001, 100)  # от 2000 до 5000 включительно, шаг 100

mde_table = [calculate_mde(p0, n, alpha, power) for n in sample_sizes]

df = pd.DataFrame({
    'Размер выборки (n)': sample_sizes,
    'MDE (абс.)': mde_table,
    'MDE (%)': [m * 100 for m in mde_table]
})

print("MDE в зависимости от размера выборки (n = 2000–5000):")
print(df.round(4))

MDE в зависимости от размера выборки (n = 2000–5000):
    Размер выборки (n)  MDE (абс.)  MDE (%)
0                 2000      0.0188   1.8794
1                 2100      0.0183   1.8341
2                 2200      0.0179   1.7919
3                 2300      0.0175   1.7525
4                 2400      0.0172   1.7156
5                 2500      0.0168   1.6810
6                 2600      0.0165   1.6483
7                 2700      0.0162   1.6175
8                 2800      0.0159   1.5883
9                 2900      0.0156   1.5607
10                3000      0.0153   1.5345
11                3100      0.0151   1.5095
12                3200      0.0149   1.4858
13                3300      0.0146   1.4631
14                3400      0.0144   1.4414
15                3500      0.0142   1.4207
16                3600      0.0140   1.4008
17                3700      0.0138   1.3817
18                3800      0.0136   1.3634
19                3900      0.0135   1.3458
20                4000